In [1]:
import pandas as pd
import numpy as np

filter = "plant.id=='<plant-id>' && hotMeasurements.doNotNormalize != true"
# tags = ['CD6:602PT038.PV','CD6:602PT047.PV','CD6:602PT073.PV']
# filter = c3.Filter().intersects("id",tags)
priority = 100
batchSize = 100
include = "id, plant.id";

### TAGS experimented:  
- `ModelId` -> Tag_name || Training Period for that Model ID 
    1. **`77a43528-4161-47cc-b3d7-834afd02fc06` -> CD6:602PC037.PV || 24 Jul '16 - 24 Jul '17**
    2. **`bf0ba412-9dc3-4d27-97e4-6e81f24fc343` -> CD6:602TT108.PV || 01 Feb '17 - 01 Feb '18**
    3. 

In [2]:
def mapper(batch, objs, job):
    import pandas as pd
    import numpy as np
    from sklearn.feature_selection import SelectKBest, chi2, f_regression, mutual_info_regression
    from scipy import stats
    from scipy.stats.stats import pearsonr
    
    #get prediction target data
    modelId = "bf0ba412-9dc3-4d27-97e4-6e81f24fc343"
    start="2017-02-01T00:00:00.000"
    end="2018-02-01T00:00:00.000"
    interval="MINUTE"
    # Fetching Target Tag Name
    predictionTargetId = c3.ValveTagModel.get(modelId, "predictionTarget.id").predictionTarget.id
    # Fetching Feature Tag names
    ids = list(map(lambda x:x.id, objs))
    
    predTargetMeasurements = c3.ValveTagModel.evalMetrics(spec={
        "ids":[modelId],
        "expressions": ["TagMeasurementsExclusionsRemoved"],
        "start":start,
        "end":end,
        "interval":interval,
        "include":"data, missing"
    })
    
    if predTargetMeasurements is not None and predTargetMeasurements.result is not None:
        predTargetMeasurements = predTargetMeasurements.result.toJson()
    else:
        predTargetMeasurements = {}
    
    #get tag data
    comparison_tag_metrics = c3.PiTag.evalMetrics(spec={
        "ids":ids,
        "expressions":["TagMeasurements"],
        "start":start,
        "end":end,
        "interval":interval
    })
    
    if comparison_tag_metrics is not None and comparison_tag_metrics.result is not None:
        comparison_tag_metrics = comparison_tag_metrics.result.toJson()
    else:
        comparison_tag_metrics = {}
    
    tag_data_target = None
    if 'm_data' in predTargetMeasurements.get(modelId, {}).get('TagMeasurementsExclusionsRemoved', {}):
        tag_data_target = predTargetMeasurements[modelId]['TagMeasurementsExclusionsRemoved']['m_data']

    # feature dataframe being populated | Started from Numpy array to dataframe
    tag_data_feature = np.empty((len(tag_data_target), len(ids)), dtype=np.float32, order='f')
    
    # To convert the c3 Obtained data to DataFrame
    for i, obj_id in enumerate(ids):
        if 'm_data' in comparison_tag_metrics.get(obj_id, {}).get("TagMeasurements", {}):
            # Using float32 to reduce the memory footprint and better performance,
            # Comparison_tag_metrics need to be modified as well, otherwise it populate the tag_data
            # with the float64
            tag_data_feature[:, i] = np.array(comparison_tag_metrics[obj_id]["TagMeasurements"]['m_data']).astype(np.float32,
                                                                                                          copy=False)
        else:
            tag_data_feature[:, i] = np.nan

   
    feature_df = pd.DataFrame(tag_data_feature,columns = ids)
    # Since we know, we already have TARGET DATAPOINTS in a numpy aaray, we directly converted it to dataframe!
    pred_df = pd.DataFrame(tag_data_target,columns=[predictionTargetId])
    
    # Running SelectKBest
    
    # Converting values in Array format for execution
    features = feature_df.values
    target = target_df.values.ravel()
    target = target.astype('int')

    # Running SelectKBest
    test = SelectKBest(score_func = f_regression, k=batch)
    fit = test.fit(features,target)
    feature_scores = fit.scores_[fit.get_support()]


    mask = test.get_support()
    new_features = feature_df.columns[mask]
    prep_df = pd.DataFrame()
    prep_df['Columns'] = feature_df.columns
    prep_df['{}_scores'.format(scoring_function.__name__)] = fit.scores_
    prep_df['P_values'] = fit.pvalues_
    out_list = []
    for column in feature_df.columns:
        corr_tuple = pearsonr(feature_df[column], target)
        out_list.append(corr_tuple[0])

    prep_df['Correlation_scores'] = out_list
    
#     def select_k_best(feature_df,pred_df,batch):
#         #feature_dataframe = feature_df.set_index('timestamp')
#         #target_dataframe = pred_df.set_index('timestamp')
        
#         # Converting values in Array format for execution
#         features = feature_df.values
#         target = target_df.values.ravel()
#         target = target.astype('int')
        
#         # Running SelectKBest
#         test = SelectKBest(score_func = f_regression, k=batch)
#         fit = test.fit(features,target)
#         feature_scores = fit.scores_[fit.get_support()]


#         mask = test.get_support()
#         new_features = feature_df.columns[mask]
#         prep_df = pd.DataFrame()
#         prep_df['Columns'] = feature_df.columns
#         prep_df['{}_scores'.format(scoring_function.__name__)] = fit.scores_
#         prep_df['P_values'] = fit.pvalues_
#         out_list = []
#         for column in feature_df.columns:
#             corr_tuple = pearsonr(feature_df[column], target)
#             out_list.append(corr_tuple[0])
        
#         prep_df['Correlation_scores'] = out_list

#         return new_features,target,prep_df

#     # Executing SelectKBest F-Regression scoring function
#     f_reg_res_df = select_k_best(feature_df,pred_df,batch)
    
#     fileName = 'features.csv'
#     url = c3.FileSystem.inst().urlFromEncodedPath(c3.FileSystem.inst().rootUrl() + "/extract/f_regression/" + fileName)
#     file = c3.FileSystem.inst().makeFile(url, "text/csv")
#     file.writeString(feature_df.to_csv())
    

#     fileName = 'target.csv'
#     url = c3.FileSystem.inst().urlFromEncodedPath(c3.FileSystem.inst().rootUrl() + "/extract/f_regression/" + fileName)
#     file = c3.FileSystem.inst().makeFile(url, "text/csv")
#     file.writeString(pred_df.to_csv())


    
    # Storing the result in a CSV @ Azure Blob
    fileName = 'f_regression_result.csv'
    url = c3.FileSystem.inst().urlFromEncodedPath(c3.FileSystem.inst().rootUrl() + "/extract/f_regression/" + fileName)
    file = c3.FileSystem.inst().makeFile(url, "text/csv")
    file.writeString(prep_df.to_csv())

          

    

In [3]:
job = c3.DynMapReduce.startFromSpec(c3.DynMapReduceSpec(
   targetType="PiTag",
   filter=filter,
   priority=priority,
   include=include,
   batchSize=batchSize,
   mapLambda=c3.Lambda.fromPython(mapper, runtime="sklearn_3_0_0")
))

In [8]:
job.status()

c3.MapReduceStatus(
 started=datetime.datetime(2022, 3, 29, 6, 20, 18, tzinfo=datetime.timezone.utc),
 startedby='atul.mishra@shell.com',
 completed=datetime.datetime(2022, 3, 29, 6, 20, 38, tzinfo=datetime.timezone.utc),
 status='completed')

##### TO check the Job status in CONSOLE:
- **`c3Grid(InvalidationQueue.countAll("type"));`**
- **`c3Grid(DynMapReduce.fetch({order: "descending(meta.created)",limit:5}))`**